Data\_Modifications

In [ ]:
-- Let's checkout our rowgroups.
SELECT  object_name(i.object_id) AS TableName,   
		i.name AS IndexName,   
		i.type_desc AS IndexType,   
		rg.state_desc AS StateDescription,
		rg.total_rows AS TotalRows,
		rg.deleted_rows AS DeletedRows,
		rg.trim_reason_desc AS TrimReason
FROM [sys].[indexes] AS i  
JOIN [sys].[dm_db_column_store_row_group_physical_stats] AS rg  
    ON i.object_id = rg.object_id AND i.index_id = rg.index_id
	WHERE i.name = 'pcci1';
GO



In [ ]:
-- Let's insert some data.
INSERT INTO Sales.SalesOrder_PCS 
SELECT TOP 500	*
FROM Sales.SalesOrder_CS;
GO


In [ ]:
-- Let's checkout our rowgroups.
SELECT  object_name(i.object_id) AS TableName,   
		i.name AS IndexName,   
		i.type_desc AS IndexType,   
		rg.state_desc AS StateDescription,
		rg.total_rows AS TotalRows,
		rg.deleted_rows AS DeletedRows,
		rg.trim_reason_desc AS TrimReason
FROM [sys].[indexes] AS i  
JOIN [sys].[dm_db_column_store_row_group_physical_stats] AS rg  
    ON i.object_id = rg.object_id AND i.index_id = rg.index_id
	WHERE i.name = 'pcci1';
GO


In [ ]:
-- Now let's update some data.
--select top 10 * from Sales.SalesOrder_PCS

UPDATE Sales.SalesOrder_PCS SET SalesAmount = SalesAmount + 1
WHERE Id <= 500;
GO

In [ ]:
-- Let's checkout our rowgroups.
SELECT  object_name(i.object_id) AS TableName,   
		i.name AS IndexName,   
		i.type_desc AS IndexType,   
		rg.state_desc AS StateDescription,
		rg.total_rows AS TotalRows,
		rg.deleted_rows AS DeletedRows,
		rg.trim_reason_desc AS TrimReason
FROM [sys].[indexes] AS i  
JOIN [sys].[dm_db_column_store_row_group_physical_stats] AS rg  
    ON i.object_id = rg.object_id AND i.index_id = rg.index_id
	WHERE i.name = 'pcci1';
GO

In [ ]:
-- Let's find out or min and max Id.
SELECT	MAX(Id) AS MaxId, 
		MIN(Id) AS MinId 
FROM Sales.SalesOrder_PCS;
GO




-- This will delete 1000 rows.
-- 500 will come from the delta store.
-- 500 will be marked as deleted
DELETE FROM Sales.SalesOrder_PCS
WHERE Id > 23800000;
GO

In [ ]:
-- Let's look at the deleted buffer.
SELECT  object_name(i.object_id) AS TableName,
		i.[name] AS IndexName, 
		p.[internal_object_type_desc] AS [Description],
		p.[rows] AS [RowCount], 
		p.[data_compression_desc] AS [CompressionType]
FROM [sys].[internal_partitions] AS p
	JOIN [sys].[indexes] AS i ON p.[object_id] = i.[object_id] 
	AND p.[index_id] = i.[index_id]
WHERE i.[name] = 'pcci1';
GO